In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
import time
from selenium.common.exceptions import NoSuchElementException


In [3]:
url = "https://www.bedca.net/bdpub/index.php"
driver = webdriver.Chrome("/usr/local/bin/chromedriver") 
driver.get(url)

In [4]:
#funciones de click

def clickOnConsulta(driver):
    consulta = driver.find_element_by_xpath("//div[@id='navigation']/div[4]")
    consulta.click()
    
def clickOnGeneral(driver):
    alfabetica = driver.find_element_by_xpath("//a[@id='General']")
    alfabetica.click()
    
def selectOption(driver,id):
    driver.find_element_by_xpath('//*[@id="fglist"]/option[{}]'.format(id)).click()
    driver.find_element_by_xpath("//*[@id='content2']/form/fieldset/input").click()
    
def clickOnElement(driver,id):
    element = driver.find_element_by_xpath("//*[@id='querytable1']/tbody/tr[{}]/td[1]/a".format(id))
    element.click()
    
def goBack(driver):
    element = driver.find_element_by_xpath("//*[@id='Todos']".format(id))
    element.click()

In [5]:
clickOnConsulta(driver)

In [6]:
clickOnGeneral(driver)

In [21]:
ignore = ['Hidratos de Carbono','Grasas','Minerales','Vitaminas']
categorias = ['Lácteos','Huevos','Carne','Pescado','Grasas y Aceites','Cereales','Legumbres','Verduras','Frutas','Azúcar','Bebidas','Otros']

def scrapeElement(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    name = getElementName(soup)
    table = soup.find_all('table')[2] #tabla de valores
    tbodys = table.find_all('tbody')[1:]
    values = []
    for tbody in tbodys:
        rows = tbody.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            saveElementValues(cols,values)
    return name, values

def getElementName(soup):
    name = soup.find("h4", { "id" : "foodname" }).get_text()
    return name
        
def saveElementValues(cols,values):
    if cols[0].get_text() not in ignore:
        values.append([col.get_text() for col in cols[:-1]])
        
def storeElement(name,values,category):
    values.append(categorias[category])
    food[name]=values

In [22]:
food = dict()

In [23]:
%%time

for category in range(1,13):
    time.sleep(1)
    selectOption(driver,category)
    rows = []
    time.sleep(1)
    rows = driver.find_elements_by_xpath("//*[@id='querytable1']/tbody/tr")
    for index in range(len(rows)-1):
        time.sleep(1)
        try:
            clickOnElement(driver,index+2)
            time.sleep(1)
        except NoSuchElementException:
            break
        name, values = scrapeElement(driver)
        if(len(values) != 40):
            print('ALERT: ',name) #Alerta si falta algun valor, los que salen en blanco es porque están completamente vacíos
        storeElement(name,values,category-1)
        goBack(driver)
    goBack(driver)

ALERT:  
ALERT:  Cabrito lechal, costillar, asado
ALERT:  Cabrito lechal, costillar, crudo
ALERT:  Cabrito lechal, espalda, cruda
ALERT:  Cabrito lechal, pierna, cruda
ALERT:  Carne de cerdo de capa blanca, cabezada, parte magra, cruda
ALERT:  Carne de cerdo de capa blanca, cabezada, partes grasa y magra, crudo
ALERT:  Carne de cerdo de capa blanca, chuleta, parte magra, cruda
ALERT:  Carne de cerdo de capa blanca, chuleta, partes grasa y magra, cruda
ALERT:  Carne de cerdo de capa blanca, jamón, parte magra, crudo
ALERT:  Carne de cerdo de capa blanca, jamón, partes grasa y magra, crudo
ALERT:  Carne de cerdo de capa blanca, lomo, parte magra, crudo
ALERT:  Carne de cerdo de capa blanca, lomo, partes grasa y magra, crudo
ALERT:  
ALERT:  
ALERT:  
ALERT:  
ALERT:  
ALERT:  
ALERT:  
ALERT:  Cordero lechal, costillar, asado
ALERT:  Cordero lechal, costillar, crudo
ALERT:  Cordero lechal, espalda, cruda
ALERT:  Cordero lechal, pierna, cruda
ALERT:  Cordero ligero, costillar, asado
ALERT

In [40]:
len(food)

958

In [39]:
with open('bedca_raw.json', 'w') as fp:
    json.dump(food, fp)